In [1]:
from bs4 import BeautifulSoup
import urllib.request
from IPython.display import HTML
import re
import pandas as pd
import requests

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

r = requests.get('https://pokemondb.net/pokedex/all', headers=HEADERS)
with open('cache.html', 'wb') as f:
    f.write(r.content)
    
with open('cache.html', 'rb') as f:
    scraped_dex = BeautifulSoup(f.read(), 'html.parser')

In [3]:
table = scraped_dex.find('table', class_='data-table sticky-header block-wide')

In [4]:
tbody = table.find('tbody')

In [5]:
td = tbody.find_all('td')

In [6]:
dexno = []

for no in td[::10]:
    dexno.append(no.find('span').string)

In [7]:
name = []
form = []

for n in td[1::10]:
    s = str(n.find('a').string)
    name.append(s)
    
    if n.find('small'):
        form.append(str(n.find('small').string))
    else:
        form.append(None)

In [8]:
type1 = []
type2 = []

for t in td[2::10]:
    type1.append(str(t.find('a').string))
    if len(t.find_all('a')) > 1:
        type2.append(str(t.find_all('a')[1].string)) 
    else:
        type2.append(None)

In [9]:
hp = []
atk = []
df = []
spatk = []
spdef = []
spd = []

for h in td[4::10]:
    hp.append(h.string)
    
for a in td[5::10]:
    atk.append(a.string)
    
for d in td[6::10]:
    df.append(d.string)
    
for sa in td[7::10]:
    spatk.append(sa.string)
    
for sd in td[8::10]:
    spdef.append(sd.string)
    
for sp in td[9::10]:
    spd.append(sp.string)

In [10]:
data = {'SPECIES_NUM': dexno, 'SPECIES_NAME' : name, 'Form': form, 'Type 1': type1, 'Type 2': type2,
        'HP': hp, 'Attack': atk, 'Defense': df,'Sp.Atk' : spatk, 'Sp.Def' : spdef, 'Speed' : spd }
df = pd.DataFrame(data)
display(df)

,SPECIES_NUM,SPECIES_NAME,Form,Type 1,Type 2,HP,Attack,Defense,Sp.Atk,Sp.Def,Speed
0,0001,Bulbasaur,None,Grass,Poison,45,49,49,65,65,45
1,0002,Ivysaur,None,Grass,Poison,60,62,63,80,80,60
2,0003,Venusaur,None,Grass,Poison,80,82,83,100,100,80
3,0003,Venusaur,Mega Venusaur,Grass,Poison,80,100,123,122,120,80
4,0004,Charmander,None,Fire,None,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...
1210,1023,Iron Crown,None,Steel,Psychic,90,72,100,122,108,98
1211,1024,Terapagos,Normal Form,Normal,None,90,65,85,65,85,60
1212,1024,Terapagos,Terastal Form,Normal,None,95,95,110,105,110,85
1213,1024,Terapagos,Stellar Form,Normal,None,160,105,110,130,110,85


In [11]:
mask = df.dropna(subset='Form')
mask_mega = ~mask['Form'].str.contains('Mega')
mask_mega = mask[~mask_mega]

In [12]:
df = df.drop(mask_mega.index)

In [13]:
df = df.set_index('SPECIES_NUM')

In [14]:
def set_region(dexno, form):
    reg_form = ['Alolan', 'Galarian', 'Hisuian', 'Paldean']
    reg = ''
    
    dexno = int(dexno)
    if form != None:
        for i in reg_form:
            if i in form:
                reg = i;
            
    if reg != '':
        if reg == reg_form[0]:
            return 6
        elif reg == reg_form[1]:
            return 7
        elif reg == reg_form[2]:
            return 8
        elif reg == reg_form[3]:
            return 9
    else:
        if dexno in range(1,152):
            return 0
        elif dexno in range(152, 252):
            return 1
        elif dexno in range(252, 387):
            return 2
        elif dexno in range(387, 494):
            return 3
        elif dexno in range(494, 650):
            return 4
        elif dexno in range(650, 722):
            return 5
        elif dexno in range(722, 810):
            return 6
        elif dexno in range(810, 899):
            return 7
        elif dexno in range(899, 906):
            return 8
        else:
            return 9
    

In [15]:
df['REGION_ID'] = [set_region(x,y) for x,y in zip(df.index, df['Form'])]

In [16]:
reg_name = ['Kanto', 'Johto', 'Hoenn', 'Sinnoh', 'Unova', 'Kalos', 'Alola', 'Galar', 'Hisui', 'Paldea']
region = { 'REGION_ID' : [x for x in range(0,10)], 'REGION_NAME' : reg_name}
region = pd.DataFrame(region).set_index('REGION_ID')
display(region)

,REGION_NAME
REGION_ID,
0,Kanto
1,Johto
2,Hoenn
3,Sinnoh
4,Unova
5,Kalos
6,Alola
7,Galar
8,Hisui


In [17]:
df['FORM_ID'] = df.groupby('SPECIES_NUM').cumcount()
form = df.drop(columns=['SPECIES_NAME','Type 1','Type 2'])
cols = form.columns.tolist()
cols = cols[-2:] + cols[:-2]
cols = cols[1:2] + cols[0:1] + cols[2:]
form = form[cols]
swap = {'Form':'FORM_NAME','HP':'FORM_BASE_HP','Attack':'FORM_BASE_ATK','Defense':'FORM_BASE_DEF',
        'Sp.Atk':'FORM_BASE_SPATK','Sp.Def':'FORM_BASE_SPDEF','Speed':'FORM_BASE_SPD'}
form = form.rename(columns=swap)
display(form)

,FORM_ID,REGION_ID,FORM_NAME,FORM_BASE_HP,FORM_BASE_ATK,FORM_BASE_DEF,FORM_BASE_SPATK,FORM_BASE_SPDEF,FORM_BASE_SPD
SPECIES_NUM,,,,,,,,,
0001,0,0,None,45,49,49,65,65,45
0002,0,0,None,60,62,63,80,80,60
0003,0,0,None,80,82,83,100,100,80
0004,0,0,None,39,52,43,60,50,65
0005,0,0,None,58,64,58,80,65,80
...,...,...,...,...,...,...,...,...,...
1023,0,9,None,90,72,100,122,108,98
1024,0,9,Normal Form,90,65,85,65,85,60
1024,1,9,Terastal Form,95,95,110,105,110,85


In [18]:
type_name = df.iloc[:,2].sort_values().unique()
types = { 'TYPE_ID' : [x for x in range(0,18)], 'TYPE_NAME' : type_name}
types = pd.DataFrame(types).set_index('TYPE_ID')
display(types)

,TYPE_NAME
TYPE_ID,
0,Bug
1,Dark
2,Dragon
3,Electric
4,Fairy
5,Fighting
6,Fire
7,Flying
8,Ghost


In [19]:
p_species = df[['SPECIES_NAME']].copy()
p_species = p_species.drop_duplicates()
display(p_species)

,SPECIES_NAME
SPECIES_NUM,
0001,Bulbasaur
0002,Ivysaur
0003,Venusaur
0004,Charmander
0005,Charmeleon
...,...
1021,Raging Bolt
1022,Iron Boulder
1023,Iron Crown


In [20]:
type1 = df[['FORM_ID','Type 1']]
rep = types.to_dict()
rep = rep['TYPE_NAME']
rep = {value: key for key, value in rep.items()}
type1 = type1.replace(rep)
type1 = type1.rename(columns={'Type 1': 'TYPE_ID'})

In [21]:
type2 = df[['FORM_ID','Type 2']]
type2 = type2.replace(rep)
type2 = type2.dropna()
type2 = type2.convert_dtypes()
type2 = type2.rename(columns={'Type 2': 'TYPE_ID'})

In [22]:
p_type = pd.concat([type1,type2])
p_type = p_type.sort_index()
display(p_type)

,FORM_ID,TYPE_ID
SPECIES_NUM,,
0001,0,9
0001,0,13
0002,0,9
0002,0,13
0003,0,9
...,...,...
1024,2,12
1024,0,12
1024,1,12


In [23]:
r2 = requests.get('https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_base_Egg_cycles', headers=HEADERS)
with open('cache.html', 'wb') as b:
    b.write(r2.content)
    
with open('cache.html', 'rb') as b:
    egg_site = BeautifulSoup(b.read(), 'html.parser')

In [24]:
table = egg_site.find('table', class_='sortable')

In [25]:
td = table.find_all('td')

In [26]:
dexno = []
egg1 = []
egg2 = []

for no in td[::12]:
    dexno.append(str(no.string).strip())
    
for g1 in td[3::12]:
    egg1.append(g1.find('a').string)
    
for g2 in td[4::12]:
    if g2.find('a') != None:
        egg2.append(g2.find('a').string)
    else:
        egg2.append(None)
        
egg = {'SPECIES_NUM' : dexno, 'Egg 1' : egg1, 'Egg 2' : egg2}
egg = pd.DataFrame(egg)
egg = egg.set_index('SPECIES_NUM')

In [27]:
gn = egg.iloc[:,0].sort_values().unique()

print(gn)
egg_group = { 'GROUP_ID' : [x for x in range(0,15)], 'GROUP_NAME' : gn}
egg_group = pd.DataFrame(egg_group)
egg_group = egg_group.set_index('GROUP_ID')

display(egg_group)

['Amorphous' 'Bug' 'Ditto' 'Dragon' 'Fairy' 'Field' 'Flying' 'Grass'
 'Human-Like' 'Mineral' 'Monster' 'No Eggs Discovered' 'Water 1' 'Water 2'
 'Water 3']


,GROUP_NAME
GROUP_ID,
0,Amorphous
1,Bug
2,Ditto
3,Dragon
4,Fairy
5,Field
6,Flying
7,Grass
8,Human-Like


In [28]:
egg1 = egg[['Egg 1']]
rep = egg_group.to_dict()
rep = rep['GROUP_NAME']
rep = {value: key for key, value in rep.items()}
egg1 = egg1.replace(rep)
egg1 = egg1.rename(columns={'Egg 1': 'GROUP_ID'})

In [29]:
egg2 = egg[['Egg 2']]
egg2 = egg2.replace(rep)
egg2 = egg2.dropna()
egg2 = egg2.convert_dtypes()
egg2 = egg2.rename(columns={'Egg 2': 'GROUP_ID'})

In [30]:
p_egg = pd.concat([egg1,egg2])
p_egg = p_egg.sort_index()
display(p_egg[-30:])

,GROUP_ID
SPECIES_NUM,
1001,11
1002,11
1003,11
1004,11
1005,11
1006,11
1007,11
1008,11
1009,11


In [31]:
r = requests.get('https://pokemondb.net/move/all', headers=HEADERS)
with open('cache.html', 'wb') as f:
    f.write(r.content)
    
with open('cache.html', 'rb') as f:
    attack_list = BeautifulSoup(f.read(), 'html.parser')

In [32]:
table = attack_list.find('table', class_='data-table sticky-header block-wide')
tbody = table.find('tbody')
td = tbody.find_all('td')

In [33]:
name=[]
mType = []
category = []
power = []
acc = []
PP = []
effect = []
prob = []

for n in td[::8]:
    s = str(n.find('a').string)
    name.append(s)
for t in td[1::8]:
    mType.append(str(t.find('a').string))
for c in td[2::8]:
    if (c.find('img')==None):
        category.append(None)
    else:
        img = c.find('img')
        category.append(img.get('alt'))
for p in td[3::8]:
    if(p.string == "—"):
        power.append(None)
    else:
        power.append(p.string)
for a in td[4::8]:
    if(a.string == "—" or a.string == "∞"):
        acc.append(None)
    else:
        acc.append(a.string)    
for P in td[5::8]:
    PP.append(p.string)
for e in td[6::8]:
    effect.append(e.string)
for probVal in td[7::8]:
    if(probVal.string == "—"):
        prob.append(None)
    else:
        prob.append(probVal.string)
    
Data = {'TYPE_ID': mType, 'CATEGORY_ID': category, 'MOVE_NAME': name, 'MOVER_PWR': power, 'MOVE_ACC': acc, 'MOVE_PP': PP,
       'MOVE_EFFECT' : effect, 'MOVE_EF_PROB': prob}

move = pd.DataFrame(Data)
display(move)

,TYPE_ID,CATEGORY_ID,MOVE_NAME,MOVER_PWR,MOVE_ACC,MOVE_PP,MOVE_EFFECT,MOVE_EF_PROB
0,Electric,Special,"10,000,000 Volt Thunderbolt",195,None,50,Pikachu-exclusive Z-Move. High critical hit ra...,None
1,Grass,Special,Absorb,20,100,50,User recovers half the HP inflicted on opponent.,None
2,Rock,Physical,Accelerock,40,100,50,User attacks first.,None
3,Poison,Special,Acid,40,100,50,May lower opponent's Special Defense.,10
4,Poison,Status,Acid Armor,None,None,50,Sharply raises user's Defense.,None
...,...,...,...,...,...,...,...,...
929,Normal,Status,Yawn,None,None,50,Puts opponent to sleep in the next turn.,None
930,Electric,Special,Zap Cannon,120,50,50,Paralyzes opponent.,100
931,Psychic,Physical,Zen Headbutt,80,90,50,May cause flinching.,20
932,Electric,Physical,Zing Zap,80,100,50,May cause flinching.,30


In [34]:
drop_word1 = "Z-Move"
drop_word2 = "Dynamax move"
drop_word3 = "G-Max Move"


mask = move['MOVE_EFFECT'].str.contains(drop_word1,case = False).fillna(False)

move = move[~mask]


mask2 = move['MOVE_EFFECT'].str.contains(drop_word2,case = False).fillna(False)

move = move[~mask2]


mask3 = move['MOVE_EFFECT'].str.contains(drop_word3,case = False).fillna(False)

move = move[~mask3]

move = move.reset_index()
move = move.drop(columns='index')
move.index.name = "MOVE_ID"

print(move.columns)
display(move)

Index(['TYPE_ID', 'CATEGORY_ID', 'MOVE_NAME', 'MOVER_PWR', 'MOVE_ACC',
       'MOVE_PP', 'MOVE_EFFECT', 'MOVE_EF_PROB'],
      dtype='object')


,TYPE_ID,CATEGORY_ID,MOVE_NAME,MOVER_PWR,MOVE_ACC,MOVE_PP,MOVE_EFFECT,MOVE_EF_PROB
MOVE_ID,,,,,,,,
0,Grass,Special,Absorb,20,100,50,User recovers half the HP inflicted on opponent.,None
1,Rock,Physical,Accelerock,40,100,50,User attacks first.,None
2,Poison,Special,Acid,40,100,50,May lower opponent's Special Defense.,10
3,Poison,Status,Acid Armor,None,None,50,Sharply raises user's Defense.,None
4,Poison,Special,Acid Spray,40,100,50,Sharply lowers opponent's Special Defense.,100
...,...,...,...,...,...,...,...,...
842,Normal,Status,Yawn,None,None,50,Puts opponent to sleep in the next turn.,None
843,Electric,Special,Zap Cannon,120,50,50,Paralyzes opponent.,100
844,Psychic,Physical,Zen Headbutt,80,90,50,May cause flinching.,20


In [35]:
cat_name = ['Physical','Special','Status']
category = {'CATEGORY_NAME':cat_name}
category = pd.DataFrame(category)
category.index.name = "CATEGORY_ID"
display(category)

,CATEGORY_NAME
CATEGORY_ID,
0,Physical
1,Special
2,Status


In [36]:
rep = category.to_dict()
rep = rep['CATEGORY_NAME']
rep = {value: key for key, value in rep.items()}
move.replace(rep, inplace=True)
display(move)

,TYPE_ID,CATEGORY_ID,MOVE_NAME,MOVER_PWR,MOVE_ACC,MOVE_PP,MOVE_EFFECT,MOVE_EF_PROB
MOVE_ID,,,,,,,,
0,Grass,1,Absorb,20,100,50,User recovers half the HP inflicted on opponent.,None
1,Rock,0,Accelerock,40,100,50,User attacks first.,None
2,Poison,1,Acid,40,100,50,May lower opponent's Special Defense.,10
3,Poison,2,Acid Armor,None,None,50,Sharply raises user's Defense.,None
4,Poison,1,Acid Spray,40,100,50,Sharply lowers opponent's Special Defense.,100
...,...,...,...,...,...,...,...,...
842,Normal,2,Yawn,None,None,50,Puts opponent to sleep in the next turn.,None
843,Electric,1,Zap Cannon,120,50,50,Paralyzes opponent.,100
844,Psychic,0,Zen Headbutt,80,90,50,May cause flinching.,20


In [37]:
rep = types.to_dict()
rep = rep['TYPE_NAME']
rep = {value: key for key, value in rep.items()}
move.replace(rep, inplace=True)
display(move)

,TYPE_ID,CATEGORY_ID,MOVE_NAME,MOVER_PWR,MOVE_ACC,MOVE_PP,MOVE_EFFECT,MOVE_EF_PROB
MOVE_ID,,,,,,,,
0,9,1,Absorb,20,100,50,User recovers half the HP inflicted on opponent.,None
1,15,0,Accelerock,40,100,50,User attacks first.,None
2,13,1,Acid,40,100,50,May lower opponent's Special Defense.,10
3,13,2,Acid Armor,None,None,50,Sharply raises user's Defense.,None
4,13,1,Acid Spray,40,100,50,Sharply lowers opponent's Special Defense.,100
...,...,...,...,...,...,...,...,...
842,12,2,Yawn,None,None,50,Puts opponent to sleep in the next turn.,None
843,3,1,Zap Cannon,120,50,50,Paralyzes opponent.,100
844,14,0,Zen Headbutt,80,90,50,May cause flinching.,20


In [38]:
types.to_csv('types.csv')
move.to_csv('move.csv')
category.to_csv('category.csv')
egg_group.to_csv('egg_group.csv')
p_egg.to_csv('p_egg.csv')
p_type.to_csv('p_type.csv')
p_species.to_csv('p_species.csv')
form.to_csv('form.csv')
region.to_csv('region.csv')